# install libraries

In [1]:
%pip install "tensorflow[and-cuda]"
%pip install opencv-python numpy pandas plotly scikit-learn matplotlib seaborn


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import cv2
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.applications import VGG16
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

2025-02-09 14:35:27.337896: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-09 14:35:27.347936: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1739090127.359629   26847 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739090127.362987   26847 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-09 14:35:27.374864: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [ ]:
# Set paths
train_path = ""
test_path = ""
validation_path = ""


CUDA available: True


In [ ]:
Function to load data
def load_data(path):
    categories = []
    images = []
    for category in os.listdir(path):
        category_path = os.path.join(path, category)
        for image in os.listdir(category_path):
            img_path = os.path.join(category_path, image)
            img = cv2.imread(img_path)
            if img is not None:  # Check if the image is loaded correctly
                img = cv2.resize(img, (224, 224))  # Resize to 224x224 for VGG16
                images.append(img)
                categories.append(category)
    return np.array(images), np.array(categories)

# Load datasets
train_images, train_labels = load_data(train_path)
test_images, test_labels = load_data(test_path)
val_images, val_labels = load_data(validation_path)


Num GPUs Available: 1
Is TensorFlow using GPU? True
Available devices:
PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')
PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


In [ ]:


# Plot sample images
fig = px.imshow(train_images[0])
fig.update_layout(title="Sample Image from Training Set")
fig.show()



In [ ]:


# Data preprocessing
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=40, width_shift_range=0.2,
                                   height_shift_range=0.2, shear_range=0.2, zoom_range=0.2,
                                   horizontal_flip=True, fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_path, target_size=(224, 224),
                                                    batch_size=32, class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(validation_path, target_size=(224, 224),
                                                        batch_size=32, class_mode='categorical')

test_generator = test_datagen.flow_from_directory(test_path, target_size=(224, 224),
                                                  batch_size=32, class_mode='categorical', shuffle=False)



In [ ]:
# Load VGG16 model and fine-tune it
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False

model = Sequential([
    base_model,
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(36, activation='softmax')
])

model.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Model training
history = model.fit(train_generator, epochs=30, validation_data=validation_generator)

In [ ]:
# Unfreeze some layers in the base model and fine-tune
base_model.trainable = True
for layer in base_model.layers[:15]:
    layer.trainable = False

model.compile(optimizer=Adam(learning_rate=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
history_fine = model.fit(train_generator, epochs=10, validation_data=validation_generator)

In [ ]:


# Model evaluation
loss, accuracy = model.evaluate(test_generator)
print(f"Test Accuracy: {accuracy*100:.2f}%")



In [ ]:
# Classification report and confusion matrix
Y_pred = model.predict(test_generator)
y_pred = np.argmax(Y_pred, axis=1)

print('Classification Report')
print(classification_report(test_generator.classes, y_pred, target_names=test_generator.class_indices.keys()))

# Create confusion matrix
cm = confusion_matrix(test_generator.classes, y_pred)

# Plot confusion matrix with annotations
cm_fig = px.imshow(cm, labels=dict(x="Predicted", y="True", color="Count"), x=list(test_generator.class_indices.keys()), y=list(test_generator.class_indices.keys()), text_auto=True)
cm_fig.update_layout(title="Confusion Matrix")
cm_fig.show()

In [ ]:
# Plot training history
fig = go.Figure()
fig.add_trace(go.Scatter(y=history.history['accuracy'], mode='lines', name='Train Accuracy'))
fig.add_trace(go.Scatter(y=history.history['val_accuracy'], mode='lines', name='Validation Accuracy'))
fig.update_layout(title='Model Accuracy', xaxis=dict(title='Epoch'), yaxis=dict(title='Accuracy'))
fig.show()

fig = go.Figure()
fig.add_trace(go.Scatter(y=history.history['loss'], mode='lines', name='Train Loss'))
fig.add_trace(go.Scatter(y=history.history['val_loss'], mode='lines', name='Validation Loss'))
fig.update_layout(title='Model Loss', xaxis=dict(title='Epoch'), yaxis=dict(title='Loss'))
fig.show()

fig_fine = go.Figure()
fig_fine.add_trace(go.Scatter(y=history_fine.history['accuracy'], mode='lines', name='Fine-tune Train Accuracy'))
fig_fine.add_trace(go.Scatter(y=history_fine.history['val_accuracy'], mode='lines', name='Fine-tune Validation Accuracy'))
fig_fine.update_layout(title='Fine-tuning Model Accuracy', xaxis=dict(title='Epoch'), yaxis=dict(title='Accuracy'))
fig_fine.show()

fig_fine_loss = go.Figure()
fig_fine_loss.add_trace(go.Scatter(y=history_fine.history['loss'], mode='lines', name='Fine-tune Train Loss'))
fig_fine_loss.add_trace(go.Scatter(y=history_fine.history['val_loss'], mode='lines', name='Fine-tune Validation Loss'))
fig_fine_loss.update_layout(title='Fine-tuning Model Loss', xaxis=dict(title='Epoch'), yaxis=dict(title='Loss'))
fig_fine_loss.show()



In [ ]:
# Display some predictions
def display_predictions(generator, model, num_images=10):
    images, labels = next(generator)
    predictions = model.predict(images)
    fig, axes = plt.subplots(1, num_images, figsize=(20, 20))
    for i in range(num_images):
        ax = axes[i]
        ax.imshow(images[i])
        true_label = list(generator.class_indices.keys())[np.argmax(labels[i])]
        pred_label = list(generator.class_indices.keys())[np.argmax(predictions[i])]
        ax.set_title(f"True: {true_label}\nPred: {pred_label}")
        ax.axis('off')
    plt.show()

display_predictions(test_generator, model)

In [ ]:
# Save the model
model.save('/kaggle/working/fruit_veg_classifier.h5')

# push to huggung face

In [ ]:
notebook_login()

In [ ]:
model_id = "ZorigClassify" #@param {type:"string"}

description = """
A model to classify the image into the thirteen arts and craft of Bhutan.
credit: @nateraw
"""
task_name = "Image Classification"
task_type = 'image-classification'
metric_name = 'Accuracy'
metric_type = 'accuracy'
metric_value = trainer.callback_metrics['val_acc'].item()

# Delete model folder, as we (re)create it here.
if Path('./model').exists():
    shutil.rmtree('./model')

token = HfFolder().get_token()
if not token:
    raise RuntimeError("You must log in to push to hub! Run notebook_login() in the cell above.")

hf_username = HfApi().whoami()['name']
model_url = HfApi().create_repo(token=token, repo_id=model_id, exist_ok=True)
model_repo = Repository("./model", clone_from=model_url, use_auth_token=token, git_email=f"{hf_username}@users.noreply.huggingface.co", git_user=hf_username)
model.save_pretrained(model_repo.local_dir)
feature_extractor.save_pretrained(model_repo.local_dir)

# Copy over tensorboard logs from lightning_logs/ into ./model/runs/
tensorboard_logs_path = next(Path(trainer.logger.log_dir).glob('events.out*'))
model_repo_logs_path = Path(model_repo.local_dir) / 'runs'
model_repo_logs_path.mkdir(exist_ok=True, parents=True)
shutil.copy(tensorboard_logs_path, model_repo_logs_path)

# Copy over a few example images
example_images_dir = Path(model_repo.local_dir) / 'images'
example_images_dir.mkdir(exist_ok=True, parents=True)
image_markdown_template = '''
#### {class_name}

![{class_name}](images/{example_image_path})
'''
example_images_markdown = ""
for class_idx, class_name in enumerate(ds.classes):
    folder = ds.root / class_name
    image_path = sorted(folder.glob('*'))[0]
    example_image_path = example_images_dir / f"{class_name.replace(' ', '_')}{image_path.suffix}"
    shutil.copy(image_path, example_image_path)
    example_images_markdown += image_markdown_template.format(
        class_name=class_name,
        example_image_path=example_image_path.name
    )


# Prepare README.md from information gathered above
readme_txt = f"""
---
tags:
- image-classification
- pytorch
- huggingpics
metrics:
- {metric_type}

model-index:
- name: {model_id}
  results:
  - task:
      name: {task_name}
      type: {task_type}
    metrics:
      - name: {metric_name}
        type: {metric_type}
        value: {metric_value}
---

# {model_id}

{description}

## Example Images

{example_images_markdown}

""".strip()

(Path(model_repo.local_dir) / 'README.md').write_text(readme_txt)

commit_url = model_repo.push_to_hub()

print("Check out your model at:")
print(f"https://huggingface.co/{hf_username}/{model_id}")